<a href="https://colab.research.google.com/github/Venuram/Machine-Learning-ML/blob/master/Basic_CNN_using_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.1.0-rc2


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images = train_images/255.0
test_images = test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning rate', values=[1e-2, 1e-3])),
                loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  return model                    

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model, objective='val_acc',  max_trials=5, directory='output', project_name = 'MNIST_fashion')

INFO:tensorflow:Reloading Oracle from output/MNIST_fashion/oracle.json


In [ ]:
tuner_search.search(train_images,train_labels,epochs=3, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 10s 193us/sample - loss: 0.4335 - acc: 0.8434 - val_loss: 0.3131 - val_acc: 0.8852
Epoch 2/3
54000/54000 [==============================] - 10s 189us/sample - loss: 0.2779 - acc: 0.8989 - val_loss: 0.2855 - val_acc: 0.8972
Epoch 3/3
54000/54000 [==============================] - 10s 191us/sample - loss: 0.2227 - acc: 0.9177 - val_loss: 0.2771 - val_acc: 0.9002


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 11s 202us/sample - loss: 0.5390 - acc: 0.8189 - val_loss: 0.4064 - val_acc: 0.8503
Epoch 2/3
54000/54000 [==============================] - 11s 199us/sample - loss: 0.4010 - acc: 0.8554 - val_loss: 0.4197 - val_acc: 0.8530
Epoch 3/3
54000/54000 [==============================] - 11s 198us/sample - loss: 0.3704 - acc: 0.8651 - val_loss: 0.4025 - val_acc: 0.8498


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 11s 196us/sample - loss: 0.5232 - acc: 0.8140 - val_loss: 0.3948 - val_acc: 0.8557
Epoch 2/3
54000/54000 [==============================] - 10s 193us/sample - loss: 0.3907 - acc: 0.8571 - val_loss: 0.3992 - val_acc: 0.8660
Epoch 3/3
54000/54000 [==============================] - 10s 192us/sample - loss: 0.3585 - acc: 0.8668 - val_loss: 0.3875 - val_acc: 0.8600


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 12s 218us/sample - loss: 0.4361 - acc: 0.8449 - val_loss: 0.3447 - val_acc: 0.8735
Epoch 2/3
54000/54000 [==============================] - 12s 217us/sample - loss: 0.3119 - acc: 0.8844 - val_loss: 0.3368 - val_acc: 0.8802
Epoch 3/3
54000/54000 [==============================] - 12s 215us/sample - loss: 0.2731 - acc: 0.8989 - val_loss: 0.3554 - val_acc: 0.8765


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 12s 219us/sample - loss: 0.3952 - acc: 0.8580 - val_loss: 0.2947 - val_acc: 0.8882
Epoch 2/3
54000/54000 [==============================] - 12s 215us/sample - loss: 0.2633 - acc: 0.9029 - val_loss: 0.2594 - val_acc: 0.9047
Epoch 3/3
54000/54000 [==============================] - 11s 209us/sample - loss: 0.2055 - acc: 0.9226 - val_loss: 0.2705 - val_acc: 0.9027


INFO:tensorflow:Oracle triggered exit


In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 128)       3328      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        55344     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2230368   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,290,010
Trainable params: 2,290,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images,train_labels,epochs=10, validation_split=0.1,initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - 11s 211us/sample - loss: 0.2034 - acc: 0.9240 - val_loss: 0.2460 - val_acc: 0.9113
Epoch 5/10
54000/54000 [==============================] - 11s 212us/sample - loss: 0.1578 - acc: 0.9416 - val_loss: 0.2704 - val_acc: 0.8987
Epoch 6/10
54000/54000 [==============================] - 11s 212us/sample - loss: 0.1202 - acc: 0.9547 - val_loss: 0.2688 - val_acc: 0.9073
Epoch 7/10
54000/54000 [==============================] - 12s 214us/sample - loss: 0.0893 - acc: 0.9669 - val_loss: 0.3005 - val_acc: 0.9145
Epoch 8/10
54000/54000 [==============================] - 11s 207us/sample - loss: 0.0679 - acc: 0.9741 - val_loss: 0.3298 - val_acc: 0.9110
Epoch 9/10
54000/54000 [==============================] - 12s 213us/sample - loss: 0.0531 - acc: 0.9805 - val_loss: 0.3796 - val_acc: 0.9088
Epoch 10/10
54000/54000 [==============================] - 12s 216us/sample - loss: 0.0395 - acc: 0.9857 